## Deploy model to an online endpoint

In [ ]:
# Import credentials
from resource_credentials import (subscription_key, 
    resource_group_name, 
    workspace_name)

from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# Create a ml client
ml_client = MLClient(credential=DefaultAzureCredential(), 
                     subscription_id=subscription_key, 
                     resource_group_name=resource_group_name,
                     workspace_name=workspace_name)

### Create endpoint

In [ ]:
from azure.ai.ml.entities import ManagedOnlineEndpoint

endpoint_name = "diabetes-model-endpoint"

endpoint = ManagedOnlineEndpoint(
    name=endpoint_name,
    auth_mode="key"
)

ml_client.begin_create_or_update(endpoint).result()

### Configure and create deployment

In [ ]:
from azure.ai.ml.entities import Model, ManagedOnlineDeployment
from azure.ai.ml.constants import AssetTypes

# Define local model
model = Model(
    path="./src/model",
    type=AssetTypes.MLFLOW_MODEL,
    name="mlflow-diabetes-model-local"
)

# Blue deployment
blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=endpoint_name,
    model=model,
    instance_type="Standard_E2s_v3",
    instance_count=1
)

ml_client.online_deployments.begin_create_or_update(blue_deployment).result()

### Test deployment

In [ ]:
# Test endpoint
response = ml_client.online_endpoints.invoke(
    endpoint_name=endpoint_name,
    deployment_name="blue",    
    request_file="./src/endpoint-test.json"
)

print(response)

if response[1]=='1':
    print("Diabetic")
else:
    print ("Not diabetic")